### **5.1 - SubDAGs - The Hard Way of Grouping your Tasks**

Ya sabes, en algún momento puedes acabar con un montón de tareas dentro de tu DAG, por lo que puede llegar a ser más y más difícil de entender exactamente lo que está pasando en tu DAG. Y tal vez una solución para hacerlo más claro y más fácil de leer podría ser agrupar las tareas que están relacionadas entre sí. Y esto es algo que se puede hacer en Airflow utilizando dos formas: los "SubDags" o los "TaskGroups". Los "TaskGroups" son definitivamente mejores, pero vamos a empezar con los SubDags. Empecemos con un ejemplo muy sencillo. Así que si haces clic en "my_dag" y vas a la Graph View, puedes ver las tres tareas process_a, _b y _c, y como están realmente relacionadas entre sí, una buena cosa podría ser agrupar esas tareas y para hacer esto vas a utilizar un SubDag. 

<center><img src="https://i.postimg.cc/Jzv7kTGh/a1221.png"></center>

Así que vamos a volver a tu editor de código y la primera pregunta que puedes tener es, ¿qué es exactamente un SubDag? Bueno, un SubDag no es más que un DAG dentro de otro DAG, y en ese SubDag, vas a poner tus tareas, las tareas que quieres agrupar. Ahora bien, para crear un SubDag necesitas dos componentes: el primero es un "Subdag Operator" y el segundo es la "factory function" encargada de crear ese SubDag, encargada de devolver el SubDag que vas a llamar desde el "SubDag operator". Así que vamos a importar primero el "Subdag Operator", así que escribe "from airflow.operators.subdag import SubdagOperator". 

<center><img src="https://i.postimg.cc/rsfMFjBM/a1222.png"></center>

Una vez que lo tengas, puedes usar ese Subdag Operator. Así que vamos a llamarlo: 

<center><img src="https://i.postimg.cc/GhpbS0D2/a1223.png"></center>

Y luego tienes que pasar el "subdag". Este argumento espera un DAG, y en ese caso vas a llamar a tu "factory function", así que vamos a llamarla "subdag_factory", con algunos argumentos que vas a ver más adelante. Así que una vez que tienes este "Subdag Operator" estás listo para usarlo. 

<center><img src="https://i.postimg.cc/66c906yg/a1224.png"></center>

El siguiente paso es crear la "factory function". Para ello puedes crear una nueva carpeta en la carpeta "dags" y llamémosla "subdag" y luego crea un nuevo archivo "subdag_factory.py". Aquí como vas a crear un nuevo DAG, necesitas importar el "objeto Dag", así que escribes "from airflow.models import DAG" y luego defines tu función, que es "subdag_factory" con algunos argumentos:

<center><img src="https://i.postimg.cc/k4dnjBN3/a1225.png"></center>

Aquí quieres pasar el "parent_dag_id" así como el "subdag_dag_id" y los "default arguments" ya que quieres asegurarte de que esos argumentos por defecto son comunes entre tu "dag padre" y tu "subdag". ¿Por qué? Lo verás en un minuto. El siguiente paso es instanciar tu "objeto Dag", así que escribes "with DAG" y aquí el "Dag ID" tiene una notación muy especial. Con el "subdag" tienes que asegurarte de que el "Dag ID" de tu "Subdag" está compuesto por el "parent_dag_id.subdag_dag_id". Así que en ese caso, pondrás: 

<center><img src="https://i.postimg.cc/JncRSMKy/a1226.png"></center>

Si no lo haces no funcionará, así que asegúrate de seguir esta notación y luego sólo tienes que decir, quiero mantener los mismos default arguments que los default arguments en el "dag padre".

<center><img src="https://i.postimg.cc/BvFScxKB/a1227.png"></center>

Una vez que su DAG es instanciado, usted está listo para poner las tareas que desea agrupar en él. Así que básicamente sólo tienes que volver a tu "dag padre", cortar esas tareas:
 
<center><img src="https://i.postimg.cc/fTgDGH7v/a1228.png"></center>

Y pegarlas en el archivo "subdag_factory.py". 

<center><img src="https://i.postimg.cc/Vvfz8pHK/a1229.png"></center>

Obviamente, usted tiene que añadir esas tareas, así como el uso de la Taskflow API, cortar desde el "padre dag" y pegarlos en el "subdag_factory.py" archivo:

<center><img src="https://i.postimg.cc/FHdNNjDX/a1230.png"></center>
<center><img src="https://i.postimg.cc/9Q6VMrnR/a1231.png"></center>

Lo mismo para la importación:

<center><img src="https://i.postimg.cc/k5WCtq0j/a1232.png"></center>

Y necesitas "partner_settings" como se muestra ahí mismo. 

<center><img src="https://i.postimg.cc/bJ8hx48q/a1233.png"></center>

Así que una cosa que puedes hacer es añadir "partner_settings" como argumento de tu función "subdag_factory". 

<center><img src="https://i.postimg.cc/9MpjJ1FZ/a1234.png"></center>

Luego, por último, pero no menos importante, necesitas devolver ese DAG, así que escribes "return dag". 

<center><img src="https://i.postimg.cc/Y0CBVghV/a1235.png"></center>

El último paso es volver a tu "dag padre" y añadir los argumentos a la función "subdag_factory", así que en ese caso quieres poner el "dag padre", que es "my_dag" y para el "subdag_dag_id" tienes que poner el task ID del Subdag Operator, sino no funcionará. Así que aquí tienes que poner "process_tasks" y luego pasas los argumentos por defecto "default_args" y "partner_settings".

<center><img src="https://i.postimg.cc/PJVnkxDq/a1236.png"></center>

Una vez hecho esto, tienes que definir los default arguments. Así que aquí, vamos a crear unos "default arguments iguales a: start date" con el valor que tienes ahí mismo:

<center><img src="https://i.postimg.cc/c1FS3DzN/a1237.png"></center>

Entonces, en lugar de tener "start_date" definida dentro del objeto Dag, puedes usar "default_args= default_args". ¿Por qué necesitas hacer esto? Bueno, porque tienes que asegurarte de que todas tus tareas comparten la misma "start_date" entre el "dag padre" y el "subdag". Una cosa que tienes que saber es que puedes tener múltiples start dates diferentes, ya que la "start_date" puede ser definida a nivel de tarea. Sé que puede ser un poco confuso en este momento, pero por lo general nunca va a hacer eso. Así que por eso aquí necesitas poner la "start_date" en el "default_args" para que puedas mantener la misma "start_date" para todas tus tareas entre el "dag padre" y el "subdag". 

<center><img src="https://i.postimg.cc/4NnTQ6CH/a1238.png"></center>

Una vez que tengas eso, el último paso es importar la "SubDag factory function", así que si escribes "from subdag.subdag_factory import subdag_factory". Guarda el archivo y ya está. 

<center><img src="https://i.postimg.cc/7Z28pqZx/a1239.png"></center>

Sin embargo, si echas un vistazo a la interfaz de usuario de Airflow, hay algo realmente interesante, de hecho, tienes un "error de importación" y si expandes ese error puedes ver que intentas establecer relaciones entre tareas en más de un DAG, entre el "sub dag" y el "parent dag". 

<center><img src="https://i.postimg.cc/BQyfYjsY/a1240.png"></center>

¿Por qué se produce este error? Pues porque utilizas la Taskflow API. Así que aquí, "partner_settings" es en realidad igual a los "XCom args" correspondientes a los diferentes XComs que has empujado, "partner_name" y "partner_path". 

<center><img src="https://i.postimg.cc/KY7X8QTw/a1241.png"></center>
<center><img src="https://i.postimg.cc/yNC4K2D0/a1242.png"></center>

Recuerda que la Taskflow API intentará hacer las dependencias automáticamente por ti. Y al pasar los "XCom args" a la función "subdag_factory".

<center><img src="https://i.postimg.cc/RFvk6fQF/a1243.png"></center>

Y así al SubDag, usted está tratando de crear relaciones, dependencias entre las tareas de su DAG y las tareas de su SubDag, y esto no es posible hacerlo en Airflow. No se pueden definir dependencias entre una tarea de un DAG y otra tarea de otro DAG. Para ello, hay que utilizar las Dag dependencies, pero esto es otro tema. Así que el punto importante aquí es, que usted no es capaz de utilizar la Taskflow API a través de la SubDag, o al menos a través del "dag padre" y el "subdag". Así que vamos a arreglar esto. Primero, elimine "partner_settings":

<center><img src="https://i.postimg.cc/vBXpWB3v/a1244.png"></center>

Y vuelve al archivo "subdag_factory.py", y ahora ya no vas a usar "partner_settings" aquí, así que puedes eliminar esos parámetros

<center><img src="https://i.postimg.cc/ZnRXjSQz/a1245.png"></center>
<center><img src="https://i.postimg.cc/mk5J5nxz/a1246.png"></center>

Así como aquí:

<center><img src="https://i.postimg.cc/T27z1Kyd/a1247.png"></center>
<center><img src="https://i.postimg.cc/X7rTCnFw/a1248.png"></center>

Y ahora si quieres extraer (pull) los XComs, necesitas usar otra función:

<center><img src="https://i.postimg.cc/pX33Q3Y3/a1249.png"></center>

Una vez que tienes esa función, puedes llamarla aquí, así que escribes "ti = get_current_context( )" y luego quieres obtener la Task instance object. 

<center><img src="https://i.postimg.cc/8zW9ZwZc/a1250.png"></center>

Así que lo obtienes allí y tienes que usar "xcom_pull". Así que escribes:

<center><img src="https://i.postimg.cc/hGBw6xYr/a1251.png"></center>

De aquí tomaron "extract_partners", para que quede más claro:

<center><img src="https://i.postimg.cc/bv9Mhrcj/a1252.png"></center>

Y por último, pero no menos importante, tienes que especificar también el DAG ID, porque técnicamente vas a extraer ese XCom dentro de otro DAG. ¿OK? Así que eso es lo que puedes ver aquí: 

<center><img src="https://i.postimg.cc/gJc7kpmL/a1253.png"></center>

Así que tienes que definir el DAG ID también, donde ese XCom ha sido empujado, y en ese caso, tienes que definir "my_dag". 

<center><img src="https://i.postimg.cc/LXybJW8C/a1254.png"></center>

Ok, sé que es un poco complicado ahora, pero así es como funciona, eso es lo que tienes que hacer si quieres mezclar la Taskflow API con el Subdag Operator. Así que una vez que tengas eso, escribe "partner_name". Lo mismo para "partner_path".

<center><img src="https://i.postimg.cc/mkt6CScb/a1255.png"></center>

Ok, entonces hagamos lo mismo para "process_b( )" y "process_c( )". Luego quitamos "partner_settings" y ya está:

<center><img src="https://i.postimg.cc/xTN4dtD5/a1256.png"></center>

Así que es un montón de trabajo. Así que como puedes ver, realmente, tienes que hacer mucho trabajo sólo para agrupar tareas y aún más trabajo si quieres usar la Taskflow API entre el "dag padre" y tu "subdag". Así que ahora, vamos a ver lo que se obtiene en la interfaz de usuario de Airflow. Haga clic en "my_dag", vaya a la Graph View y esta vez obtendrá sólo una tarea "process_task" que es de hecho el "Subdag Operator"

<center><img src="https://i.postimg.cc/zv62jRWT/a1257.png"></center>

Si haces clic en él, y haces zoom en el subdag, obtienes las tareas process_a, _b y _c. 

<center><img src="https://i.postimg.cc/HkGP8XTg/a1258.png"></center>
<center><img src="https://i.postimg.cc/qv8Dsq7C/a1259.png"></center>

Muy bien, hay quizás una última cosa que puedes hacer, es definir las dependencias entre "extract_partners" y "process_tasks". 

<center><img src="https://i.postimg.cc/QNyzsvPV/a1260.png"></center>

Así que vamos a volver a su editor de código y escribe:

<center><img src="https://i.postimg.cc/25rc2Yq9/a1261.png"></center>

Vuelve al archivo "subdag_factory.py" y pon "task_ids" de lo contrario te dará un error. 

<center><img src="https://i.postimg.cc/xd5sZZMf/a1262.png"></center>

Guarda el archivo y ahora si vuelves a la interfaz de Airflow, obtendrás el siguiente data pipeline con "extract_partners" y "process_tasks". 

<center><img src="https://i.postimg.cc/SsKZbGfW/a1263.png"></center>

Vamos a activar el toggle del DAG para ver si funciona. Así que actualiza la página, luego clic sobre Graph View, "extract_partners" se está ejecutando, así como el subdag, haga clic en "process_tasks", a continuación, "zoom into subdag" 

<center><img src="https://i.postimg.cc/FRnTrJTw/a1264.png"></center>

vaya a Graph view, y podrá ver que todas las tareas se han ejecutado con éxito:

<center><img src="https://i.postimg.cc/QC90HxPC/a1265.png"></center>
<center><img src="https://i.postimg.cc/Y058gBK3/a1266.png"></center>

Si haces clic en "process_a" y luego vas a "Log", obtienes como puedes ver el "partner_name" así como el "partner_path". 

<center><img src="https://i.postimg.cc/wx1ph5Yy/a1267.png"></center>

Es realmente un montón de trabajo sólo para agrupar las tareas, y es por eso que los "Task groups" son mucho mejor. Antes de pasar a esta nueva y sorprendente característica, me gustaría darte alguna información adicional sobre los SubDags. En primer lugar, tienes que saber que el "Subdag Operator" es un sensor detrás de la escena, lo que significa que espera a que las tareas dentro del SubDag se completen antes de avanzar y también puedes especificar, por ejemplo, un "poke_interval", aquí, por ejemplo, quieres comprobar si las tareas dentro del SubDag se completan cada 15 segundos, entonces puedes poner "poke_interval=15". 

<center><img src="https://i.postimg.cc/y62znJ7f/a1268.png"></center>

También para evitar acabar con el bloqueo, puedes usar el "mode reschedule". 

<center><img src="https://i.postimg.cc/L6fFWHXm/a1269.png"></center>

Y si te preguntas de qué estoy hablando, descubrirás esto sobre los sensores, pero ten en cuenta que el "Subdag Operator" es de hecho un sensor detrás de la escena, lo que no era el caso antes de Airflow 2.0. 

<center><img src="https://i.postimg.cc/t4jGQLZW/a1270.png"></center>

Por último, pero no menos importante, si quieres controlar la concurrencia de tus tareas, como vas a ver más adelante, no pongas el parámetro en el "Operador SubDag" "task_concurrency". Tendrás que ponerlo para cada una de tus tareas dentro del SubDag, de lo contrario no funcionará. Ahora es el momento de descubrir los "Task groups" que son definitivamente mejores que los SubDags.